In [1]:
import os
import numpy as np
import pandas as pd

from exp_spec_info import *
from plot_info import *
from select_data import *

In [2]:
# Data pickle path
extracted_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extracted_data.pkl"

# Processed data paths
extended_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extended_data.pkl"
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

##### Extend Data with Processed Elements

In [3]:
data = pd.read_pickle(extracted_data_path)
rel_time_ext = []
rel_res_ext = []

for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:
            for exp_iter in range(N_EXPERIMENT_ITERATIONS):

                sub_data = df_sel_setup_matrix_restart(
                    data, setup, matrix, restart_param
                )
                sub_data = sub_data[sub_data["experiment_iter"] == exp_iter]
                
                # Process data for relres extension
                rel_res = sub_data["inner_relres"].apply(
                    lambda relres_arr: relres_arr[-1]
                )
                if len(rel_res) > 0:
                    rel_res_ext.append(rel_res)

                # Process data for relative time extension
                if sub_data[sub_data["solver"] == "FP FP64"].size != 0:

                    fp64_time = sub_data[
                        sub_data["solver"] == "FP FP64"
                    ].iloc[0]["elapsed_time_ms"]
                    rel_time = sub_data["elapsed_time_ms"]/fp64_time
                    rel_time_ext.append(rel_time)

rel_time_ext = pd.concat(rel_time_ext)
rel_time_ext.name = "rel_time"

rel_res_ext = pd.concat(rel_res_ext)
rel_res_ext.name = "final_relres"

extended_data = data.join((rel_time_ext, rel_res_ext))
display(extended_data)

extended_data.to_pickle(extended_data_path)

,setup,matrix,solver,restart_param,experiment_iter,populated,initiated,converged,terminated,outer_iters,inner_iters,elapsed_time_ms,HS_trans_outer_iter,HS_trans_inner_iter,SD_trans_outer_iter,SD_trans_inner_iter,outer_relres,inner_relres,rel_time,final_relres
16560,unpreconddense,af23560,FP FP16,10,0,True,True,False,True,1500,15000,162396,-1,-1,-1,-1,"[1.0, 0.26191688735114216, 0.1433399679231576,...","[1.0, 0.7654300510340972, 0.6727755341655796, ...",0.692113,3.336230e-02
16561,unpreconddense,af23560,FP FP16,10,1,True,True,False,True,1500,15000,162822,-1,-1,-1,-1,"[1.0, 0.2685580552433235, 0.1453978649226777, ...","[1.0, 0.757513762983052, 0.6779277393820136, 0...",0.694213,3.375558e-02
16562,unpreconddense,af23560,FP FP16,10,2,True,True,False,True,1500,15000,163339,-1,-1,-1,-1,"[1.0, 0.27748500207574484, 0.14432843943042656...","[1.0, 0.7524090915924405, 0.6675266546952295, ...",0.696450,3.390211e-02
16584,unpreconddense,af23560,FP FP32,10,0,True,True,False,True,1500,15000,184934,-1,-1,-1,-1,"[1.0, 0.2619283153801638, 0.1433397260418014, ...","[1.0, 0.7654423525881157, 0.6728151381900123, ...",0.788167,3.277221e-02
16585,unpreconddense,af23560,FP FP32,10,1,True,True,False,True,1500,15000,184287,-1,-1,-1,-1,"[1.0, 0.26859348089169455, 0.1454388179350554,...","[1.0, 0.7575270843166467, 0.6779693673676339, ...",0.785731,3.255271e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36472,unprecond,Zhao2,PC HSD S2T,100,1,True,True,True,True,39,3801,6639,20,1802,21,1902,"[1.0, 0.006864433302234795, 0.0028329247375971...","[1.0, 0.9796197736458981, 0.44424981435155075,...",1.036372,5.374752e-13
36473,unprecond,Zhao2,PC HSD S2T,100,2,True,True,True,True,38,3701,6499,21,1902,22,2002,"[1.0, 0.006342051508492427, 0.0026946036825350...","[1.0, 0.9768609991788817, 0.44734658205822686,...",1.047888,6.276719e-13
36474,unprecond,Zhao2,PC HSD S2T,150,0,True,True,True,True,27,3901,7705,16,2102,17,2252,"[1.0, 0.003663027921936122, 0.0018994116795691...","[1.0, 0.9791277363805698, 0.4488118616729598, ...",1.028019,6.905619e-13
36475,unprecond,Zhao2,PC HSD S2T,150,1,True,True,True,True,27,3901,7718,16,2102,17,2252,"[1.0, 0.0035827563693714664, 0.001680961968812...","[1.0, 0.9798814359645774, 0.4454125543127461, ...",1.031818,8.499350e-13


##### Compress into Experiment Iteration Median Values

In [4]:
median_data = []

for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:

            sub_data = df_sel_setup_matrix_restart(
                extended_data, setup, matrix, restart_param
            )

            for solver in SOLVERS:

                solve_experiments = sub_data[sub_data["solver"] == solver]

                if len(solve_experiments) > 0:

                    median_experiment = {
                        "setup": setup,
                        "matrix": matrix,
                        "solver": solver,
                        "restart_param": restart_param,
                        "all_populated": bool(solve_experiments["populated"].all()),
                        "all_initiated": bool(solve_experiments["initiated"].all()),
                        "all_converged": bool(solve_experiments["converged"].all()),
                        "all_terminated": bool(solve_experiments["terminated"].all()),
                        "med_outer_iter": solve_experiments["outer_iters"].median(),
                        "med_inner_iter": solve_experiments["inner_iters"].median(),
                        "med_inner_iter": solve_experiments["inner_iters"].median(),
                        "med_rel_time": solve_experiments["rel_time"].median(),
                        "med_rel_res": solve_experiments["final_relres"].median()
                    }

                    median_data.append(median_experiment)

median_data = pd.DataFrame(median_data)
display(median_data)

median_data.to_pickle(median_data_path)

,setup,matrix,solver,restart_param,all_populated,all_initiated,all_converged,all_terminated,med_outer_iter,med_inner_iter,med_rel_time,med_rel_res
0,unprecond,af_0_k101,PC HSD RRT,10,True,True,False,True,1500.0,15000.0,0.759726,0.000005
1,unprecond,af_0_k101,PC SD CS,10,True,True,False,True,1500.0,15000.0,0.760301,0.000005
2,unprecond,af_0_k101,FP FP64,10,True,True,False,True,1500.0,15000.0,1.000000,0.000005
3,unprecond,af_0_k101,PC SD ORC,10,True,True,False,True,1500.0,15000.0,0.855195,0.000005
4,unprecond,af_0_k101,FP FP32,10,True,True,False,True,1500.0,15000.0,0.761818,0.000005
...,...,...,...,...,...,...,...,...,...,...,...,...
2723,unpreconddense,Zhao2,PC SD ORC,10,True,True,False,True,1500.0,15000.0,0.888398,0.003332
2724,unpreconddense,Zhao2,FP FP32,10,True,True,False,True,1500.0,15000.0,0.815890,0.003332
2725,unpreconddense,Zhao2,PC SD RRT,10,True,True,False,True,1500.0,15000.0,0.817055,0.003332
2726,unpreconddense,Zhao2,PC HSD ORC,10,True,True,False,True,1500.0,15000.0,0.861105,0.003511
